In [230]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [231]:
df = pd.read_csv('main.csv')

In [232]:
df = df.drop(columns=['Collector ID', 'IP Address', 'Email Address', 'Custom Data 1','Respondent ID','First Name','Last Name'])
print(df.isnull().sum())

Start Date                                                        1
End Date                                                          1
I identify as having a mental illness                             0
Education                                                         0
I have my own computer separate from a smart phone                0
I have been hospitalized before for my mental illness             0
How many days were you hospitalized for your mental illness      37
I am currently employed at least part-time                        0
I am legally disabled                                             0
I have my regular access to the internet                          0
I live with my parents                                            0
I have a gap in my resume                                         0
Total length                                                      0
Annual income (including any social welfare programs) in USD      0
I am unemployed                                 

In [233]:
df.rename(columns={'Unnamed: 28': 'Anxiety'}, inplace=True)
df.rename(columns={'Unnamed: 28': 'Anxiety'}, inplace=True)
df.rename(columns={'Unnamed: 29': 'Depression'}, inplace=True)
df.rename(columns={'Unnamed: 31': 'Mood Swings'}, inplace=True)
df.rename(columns={'Unnamed: 32': 'Panic attacks'}, inplace=True)
df.rename(columns={'Unnamed: 33': 'Compulsive behavior'}, inplace=True)
df.rename(columns={'Unnamed: 30': 'Obsessive thinking'}, inplace=True)
df.rename(columns={'Unnamed: 34': 'Tiredness'}, inplace=True)

In [234]:
df['Anxiety'].fillna(0, inplace=True)
df['Anxiety'] = df['Anxiety'].apply(lambda x: 1 if x != 0 else 0)
df['Depression'].fillna(0, inplace=True)
df['Depression'] = df['Depression'].apply(lambda x: 1 if x != 0 else 0)
df['Mood Swings'].fillna(0, inplace=True)
df['Mood Swings'] = df['Mood Swings'].apply(lambda x: 1 if x != 0 else 0)
df['Panic attacks'].fillna(0, inplace=True)
df['Panic attacks'] = df['Panic attacks'].apply(lambda x: 1 if x != 0 else 0)
df['Compulsive behavior'].fillna(0, inplace=True)
df['Compulsive behavior'] = df['Compulsive behavior'].apply(lambda x: 1 if x != 0 else 0)
df['Obsessive thinking'].fillna(0, inplace=True)
df['Obsessive thinking'] = df['Obsessive thinking'].apply(lambda x: 1 if x != 0 else 0)
df['Tiredness'].fillna(0, inplace=True)
df['Tiredness'] = df['Tiredness'].apply(lambda x: 1 if x != 0 else 0)
df['Lack of concentration'].fillna(0, inplace=True)
df['Lack of concentration'] = df['Lack of concentration'].apply(lambda x: 1 if x != 0 else 0)

In [235]:
df = df.iloc[1:]

In [236]:
df.rename(columns={'I identify as having a mental illness':'MentallyIll'}, inplace =True)
df.rename(columns={'I am unemployed':'Unemployed'}, inplace =True)
df.rename(columns={'Annual income (including any social welfare programs) in USD':'Annual'}, inplace =True)

In [237]:
excluded_values = ['0', '35', '65', '20', '78', '44', '60', '13', '8', '1', '91', '14', '5', '21', '99', '2', '6', '100', '42', '3', '28', '15', '10', '27', '43']
df['How many days were you hospitalized for your mental illness'] = pd.to_numeric(df['How many days were you hospitalized for your mental illness'], errors='coerce')
filtered_values = df[~df['How many days were you hospitalized for your mental illness'].isin(excluded_values)]
mean_hospitalization_days = filtered_values['How many days were you hospitalized for your mental illness'].mean()
print("Среднее количество дней госпитализации (без учета исключенных значений):", mean_hospitalization_days)

Среднее количество дней госпитализации (без учета исключенных значений): 3.276094276094276


In [238]:
top_two_incomes = df[df['Household Income'] != 'Prefer not to answer']['Household Income'].value_counts().index[:3]

In [239]:
df = df.replace(to_replace=['No', 'Yes'], value=[0, 1])

In [240]:
df['Start Date'] = pd.to_datetime(df['Start Date'])
df['End Date'] = pd.to_datetime(df['End Date'])
df['duration'] = df['End Date'] - df['Start Date']

In [241]:
df = df.drop(['Start Date', 'End Date'], axis=1)

In [242]:
df['duration'] = pd.to_timedelta(df['duration'])
df['duration_seconds'] = df['duration'].dt.total_seconds()

In [243]:
df = df.drop(columns=['duration'])

In [244]:
mask = df['MentallyIll'] == 0
df.loc[mask, 'How many days were you hospitalized for your mental illness'] = df.loc[mask, 'How many days were you hospitalized for your mental illness'].fillna(0)

In [245]:
df.isnull().sum()

MentallyIll                                                     0
Education                                                       0
I have my own computer separate from a smart phone              0
I have been hospitalized before for my mental illness           0
How many days were you hospitalized for your mental illness     5
I am currently employed at least part-time                      0
I am legally disabled                                           0
I have my regular access to the internet                        0
I live with my parents                                          0
I have a gap in my resume                                       0
Total length                                                    0
Annual                                                          0
Unemployed                                                      0
I read outside of work and school                               0
Annual income from social welfare programs                      0
I receive 

In [246]:
filtered_df = df[(df['How many days were you hospitalized for your mental illness'].isnull()) | 
                 (df['Region'].isnull()) |
                 (df['Household Income'] == 'Prefer not to answer') |
                 (df['Device Type'] == 'Other')]
df.drop(filtered_df.index, inplace=True)
filtered_df.to_csv('null_data.csv', index=False)

In [247]:
df1=pd.read_csv("null_data.csv")

In [248]:
df1['Device Type'] = df1['Device Type'].replace('Other', 'Windows Desktop / Laptop')

In [249]:
region_counts = df['Region'].value_counts()
most_common_region = region_counts.idxmax()
df1['Region'].fillna(most_common_region, inplace=True)

In [209]:
most_common_region 

'South Atlantic'

In [250]:
df1["Region"].isnull().value_counts()

False    46
Name: Region, dtype: int64

In [251]:
df1['How many days were you hospitalized for your mental illness'].fillna(mean_hospitalization_days, inplace=True)

In [252]:
import random
probabilities = [0.6, 0.4,0.1]
for index, row in df1.iterrows():
    if row['Household Income'] == 'Prefer not to answer':
        replacement_value = random.choices(top_two_incomes, weights=probabilities)[0]
        df1.at[index, 'Household Income'] = replacement_value

In [253]:
df['Household Income'] = df['Household Income'].str.replace('$', '', regex=True).str.replace(',', '', regex=True)

df['Household Income'] = df['Household Income'].str.replace('+', '-', regex=True)
df[['Lower_Household_Income', 'Upper_Household_Income']] = df['Household Income'].str.split('-', expand=True)

df['Lower_Household_Income'] = pd.to_numeric(df['Lower_Household_Income'], errors='coerce')
df['Upper_Household_Income'] = pd.to_numeric(df['Upper_Household_Income'], errors='coerce')

df['Upper_Household_Income'].fillna(999999, inplace=True)
df.drop('Household Income', axis=1, inplace=True)


In [254]:
df1['Household Income'] = df1['Household Income'].str.replace('$', '', regex=True).str.replace(',', '', regex=True)

df1['Household Income'] = df1['Household Income'].str.replace('+', '-', regex=True)
df1[['Lower_Household_Income', 'Upper_Household_Income']] = df1['Household Income'].str.split('-', expand=True)

df1['Lower_Household_Income'] = pd.to_numeric(df1['Lower_Household_Income'], errors='coerce')
df1['Upper_Household_Income'] = pd.to_numeric(df1['Upper_Household_Income'], errors='coerce')

df1['Upper_Household_Income'].fillna(999999, inplace=True)
df1.drop('Household Income', axis=1, inplace=True)


In [255]:
df.isnull().sum()

MentallyIll                                                     0
Education                                                       0
I have my own computer separate from a smart phone              0
I have been hospitalized before for my mental illness           0
How many days were you hospitalized for your mental illness     0
I am currently employed at least part-time                      0
I am legally disabled                                           0
I have my regular access to the internet                        0
I live with my parents                                          0
I have a gap in my resume                                       0
Total length                                                    0
Annual                                                          0
Unemployed                                                      0
I read outside of work and school                               0
Annual income from social welfare programs                      0
I receive 

In [256]:
df = pd.get_dummies(df, columns=['Education', 'Region','Gender','Age','Device Type'])

In [257]:
df1 = pd.get_dummies(df1, columns=['Education', 'Region','Gender','Age','Device Type'])

In [258]:
from sklearn.model_selection import train_test_split
stratify_column = 'Unemployed'
train_df, test_df = train_test_split(df, test_size=0.35, stratify=df[stratify_column], random_state=42)
train_df.to_csv('train_data.csv', index=False)
test_df.to_csv('test_data.csv', index=False)

In [259]:
train_df=pd.read_csv("train_data.csv")

In [260]:
merged_df = pd.concat([df1, train_df], ignore_index=True)

In [261]:
merged_df.fillna(0, inplace=True)

In [262]:
merged_df.to_csv('trainschybou.csv', index=False)

In [263]:
merged_df['Unemployed'].value_counts()

0    171
1     62
Name: Unemployed, dtype: int64